In [31]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

#Navigate to /mydrive/yolov4
%cd /mydrive/DataScienceProject/Project/CameraCalibration/

# list the contents of /mydrive
!ls

/content/gdrive/MyDrive/DataScienceProject/Project
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
/content/gdrive/My Drive/DataScienceProject/Project/CameraCalibration
cali2		   caliResult1.png   caliResultC1.png  intrinsics.xml
cali3		   caliResult2.png   caliResultC2.png  uncali2
calib.ipynb	   caliResultA1.png  intrinsics2.xml   uncali3
calibrated_images  caliResultA2.png  intrinsics3.xml   uncalibrated_images


In [39]:
images = glob.glob('uncalibrated_images/*.png')
count = 1
for image in images:
  img = cv2.imread(image)
  print(img.shape)

(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)
(640, 480, 3)


In [45]:
import numpy as np
import cv2 
import glob
from google.colab.patches import cv2_imshow

################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (20,14)
frameSize = (640,480)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
a = 9
b = 6
objp = np.zeros((b*a,3), np.float32)
objp[:,:2] = np.mgrid[0:a,0:b].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('uncalibrated_images/*.png')
count = 1
for image in images:
  img = cv2.imread(image)

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Find the chess board corners
  ret, corners = cv2.findChessboardCorners(gray, (a,b), None)
  print(image, ret)

  # If found, add object points, image points (after refining them)
  if ret == True:
    objpoints.append(objp)

    cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints.append(corners)

    # Draw and display the corners
    cv2.drawChessboardCorners(img, (a,b), corners, ret)
    cv2_imshow(img)
    cv2.imwrite(f'calibrated_images/data' + str(count) + '.png', img)
    count = count +1




Output hidden; open in https://colab.research.google.com to view.

In [46]:
############## CALIBRATION #######################################################

ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

# persist intrinsics and distortions
fs = cv2.FileStorage("intrinsics.xml", cv2.FileStorage_WRITE)
fs.write("image_width", gray.shape[1])
fs.write("image_height", gray.shape[0])
fs.write("camera_matrix", cameraMatrix)
fs.write("distortion_coefficients", dist)
fs.release()

In [47]:
############## UNDISTORTION #####################################################
img = cv2.imread('cali3/data1.png')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

print("cameraMatrix")
print(cameraMatrix)
print("\nnewCameraMatrix")
print(newCameraMatrix)
# Undistort
dst = cv2.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('caliResulttt1.png', dst)

# Undistort with Remapping
mapx, mapy = cv2.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('caliResulttt2.png', dst)

# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )

cameraMatrix
[[2.01081083e+03 0.00000000e+00 3.21927621e+02]
 [0.00000000e+00 1.70268032e+03 2.36931387e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

newCameraMatrix
[[1.94675159e+03 0.00000000e+00 3.77616704e+02]
 [0.00000000e+00 1.70268030e+03 2.36931378e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
total error: 0.658677152993885


In [37]:
aa = 10 * (2.46845532e+03 / 1.57888879e+03)
aa

15.63413038102576

In [ ]:
####OLD
cameraMatrix
[[2.01081083e+03 0.00000000e+00 3.21927621e+02]
 [0.00000000e+00 1.70268032e+03 2.36931387e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

newCameraMatrix
[[1.62630408e+03 0.00000000e+00 3.44454853e+02]
 [0.00000000e+00 9.34127563e+02 1.06203430e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
total error: 0.658677152993885